In [6]:
import os, sys
currentFolder = os.path.abspath('')
try:
    sys.path.remove(str(currentFolder))
except ValueError: # Already removed
    pass

projectFolder = 'C:/Users/abjawad/Documents/GitHub/local-attention-model'
sys.path.append(str(projectFolder))
os.chdir(projectFolder)
print( f"current working dir{os.getcwd()}")

current working dirC:\Users\abjawad\Documents\GitHub\local-attention-model


In [7]:

import torch
from  torch.utils.data import DataLoader
import torch.nn as nn
from models.builder import EncoderDecoder as segmodel
from dataloader.cfg_defaults import get_cfg_defaults
from utils.lr_policy import WarmUpPolyLR
from utils.init_func import init_weight, group_weight
from config_cityscapes import *
import yaml
import os
from dataloader.cityscapes_dataloader import CityscapesDataset

In [8]:
config_path = 'C:/Users/abjawad/Documents/GitHub/local-attention-model/dataloader/cityscapes_rgbd_config.yaml'
# with open(config_path) as info:
#     info_dict = yaml.load(info, Loader=yaml.FullLoader)
cfg = get_cfg_defaults()
cfg.merge_from_file(config_path)
cfg.freeze()

data_mean = [0.291,  0.329,  0.291]
data_std = [0.190,  0.190,  0.185]

In [9]:
cityscapes_train = CityscapesDataset(cfg, split='train')
train_loader = DataLoader(cityscapes_train, batch_size=4, shuffle=True, num_workers=4, drop_last=True)
print(f'total train: {len(cityscapes_train)} t_iteration:{len(train_loader)}')

cityscapes_val = CityscapesDataset(cfg, split='val')
val_loader = DataLoader(cityscapes_val, batch_size=1, shuffle=False, num_workers=4)
print(f'total val: {len(cityscapes_val)} v_iteration:{len(val_loader)}')

Using RGB input
Using RGB input
Found 2975 train images
total train: 2975 t_iteration:743
Using RGB input
Using RGB input
Found 500 val images
total val: 500 v_iteration:500


#### Visualiz Image 

In [10]:
import matplotlib.pyplot as plt
import numpy as np

# Access the first batch from the DataLoader
for batch in train_loader:
    images = batch['image']

    # Process each image in the batch
    for i in range(images.shape[0]):
        img = images[i].numpy().transpose((1, 2, 0))
        img *= data_std
        img += data_mean

        # print(img.shape)
    #     # Plot the image in the subplot
    #     plt.subplot(1, 4, i + 1)  # 1 row, 4 columns, i+1 is the current subplot
    #     plt.imshow(img)
    #     plt.axis('off')

    # plt.show()

    # Break the loop after the first batch
    break


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\abjawad\.conda\envs\pt_cuda\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\abjawad\.conda\envs\pt_cuda\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\abjawad\.conda\envs\pt_cuda\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\abjawad\Documents\GitHub\local-attention-model\dataloader\cityscapes_dataloader.py", line 57, in __getitem__
    sample = self.loader.load_sample(img_path, depth_path, lbl_path)
  File "C:\Users\abjawad\Documents\GitHub\local-attention-model\dataloader\cityscapes_dataloader_utils\SampleLoader.py", line 53, in load_sample
    sample = self.transform_tr

In [ ]:
# write some error to stop notebook at this point
assert False, "Stop notebook here"

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=config.background)
BatchNorm2d = nn.BatchNorm2d
model=segmodel(cfg=config, criterion=criterion, norm_layer=BatchNorm2d)

params_list = []
params_list = group_weight(params_list, model, BatchNorm2d, config.lr)        
optimizer = torch.optim.AdamW(params_list, lr=config.lr, betas=(0.9, 0.999), weight_decay=config.weight_decay)
total_iteration = config.nepochs * config.niters_per_epoch
lr_policy = WarmUpPolyLR(config.lr, config.lr_power, total_iteration, config.niters_per_epoch * config.warm_up_epoch)
print(f'lr_policy:{vars(lr_policy)}')



for epoch in range(1, 2):
    model.train()
    optimizer.zero_grad()
    sum_loss = 0
    m_iou_batches = 0
    for idx, sample in enumerate(train_loader):
        imgs = sample['image']
        gts = sample['label']
        imgs = imgs.to(f'cuda:{model.device_ids[0]}', non_blocking=True)
        gts = gts.to(f'cuda:{model.device_ids[0]}', non_blocking=True)  

        print('images shape', imgs.shape)
        print('gts shape', gts.shape)
        # loss, out = model(imgs, gts)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        # sum_loss += loss
        # print(f'epoch:{epoch} iteration:{idx} loss:{loss}')
        break
    # print(f'epoch:{epoch}')